<a href="https://colab.research.google.com/github/korupalli/ML-Project/blob/master/balancing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from tqdm import tqdm

In [3]:
frame = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data.csv')
frame.Mentions.replace(np.NaN,0,inplace=True)

In [11]:
frame.head()

,Tweet Id,Followers,Friends,Retweets,Favorites,Mentions,Hashtags,Day,Month,Date,Time,Year,Pos,Neg
0,1178791985106153472,36365,19344,13,17,0,NaN,Mon,Sep,30,22,2019,1,-4
1,1178798309491822592,9009,4943,26,89,0,VFS,Mon,Sep,30,22,2019,2,-1
2,1178803487037108225,156538,3858,56,131,0,Raiders,Mon,Sep,30,22,2019,1,-1
3,1178803977376415749,27014,24488,39,37,AOC,NaN,Mon,Sep,30,22,2019,2,-1
4,1178813521884172288,39568,37514,136,140,0,NaN,Mon,Sep,30,23,2019,2,-1


In [6]:
frame.drop(frame.index[frame[frame['Retweets'] <= 10].index.values], inplace=True)
frame.reset_index(inplace=True)

In [10]:
#Dropping duplicated index columns
frame.drop(frame.columns[[0, 1]], axis = 1, inplace = True) 

In [39]:
frame['bin'] = pd.cut(frame['Retweets'], [10, 50, 100, 1000, 10000, 275530])

In [46]:
frame['bin'] = frame.bin.apply(preprocessing.LabelEncoder().fit_transform)

In [50]:
frame.bin.value_counts()

0    1051868
2     471878
1     358895
3      40149
4       2347
Name: bin, dtype: int64

In [51]:
df_class_0 = frame[frame['bin'] == 0]
df_class_1 = frame[frame['bin'] == 1]
df_class_2 = frame[frame['bin'] == 2]
df_class_3 = frame[frame['bin'] == 3]
df_class_4 = frame[frame['bin'] == 4]

In [55]:
df_class_0_under = df_class_0.sample(2347)
df_class_1_under = df_class_1.sample(2347)
df_class_2_under = df_class_2.sample(2347)
df_class_3_under = df_class_3.sample(2347)

In [56]:
df = pd.concat([df_class_0_under, df_class_1_under,df_class_2_under,df_class_3_under, df_class_4], axis=0)

In [60]:
frame = df

In [61]:
month_map={'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11 , 'Dec':12}
frame.Month = frame.Month.map(month_map)

day_map={'Mon':1, 'Tue':2, 'Wed':3, 'Thu':4, 'Fri':5, 'Sat':6, 'Sun':7}
frame.Day = frame.Day.map(day_map)

frame.Month = frame.Month.astype(np.object)
frame.Day = frame.Day.astype(np.object)
frame.Date = frame.Date.astype(np.object)
frame.Time = frame.Time.astype(np.object)
frame.Year = frame.Year.astype(np.object)

frame.Mentions.replace(np.NaN,0,inplace=True)
frame['Mentions_count'] = frame.Mentions.apply(lambda x:0 if x==0 else len(x.split()))

frame.Hashtags.replace(np.NaN,0,inplace=True)
frame['Hashtags_count'] = frame.Hashtags.apply(lambda x:0 if x==0 else len(x.split()))



In [65]:
frame.head()

,Tweet Id,Followers,Friends,Retweets,Favorites,Mentions,Hashtags,Day,Month,Date,Time,Year,Pos,Neg,bin,Mentions_count,Hashtags_count
731608,1236770340686069760,8676,1672,16,39,0,0,7,3,8,21,2020,3,-1,0,0,0
1355468,1251049144749154304,134387,147,18,168,0,0,5,4,17,7,2020,2,-1,0,0,0
547741,1226623357916995584,217727,230594,22,65,0,0,7,2,9,21,2020,1,-1,0,0,0
1707419,1248823243261136897,12024,175,17,53,IPRD_Bihar,BiharFightsCorona BiharCares,6,4,11,4,2020,2,-1,0,1,2
1063240,1239471429764231168,79264,76936,29,73,0,0,1,3,16,8,2020,1,-3,0,0,0


**Fing score of each row basing on mentions**

In [ ]:
mentions_count = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Mentions_count.csv')
mentions_count.set_index('Unnamed: 0', inplace=True)
mentions_count.head()

In [69]:
def funct(string):
  if string == 0:
    return 0
  final = 0
  for i in string.split():
    final += mentions_count.loc[i].values[0]
  #print(string.split(), final)
  return final

In [ ]:
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook().pandas(desc="Example Desc")

frame['Mentions_score'] = frame['Mentions'].progress_apply(funct)

In [75]:
frame.head()

,Tweet Id,Followers,Friends,Retweets,Favorites,Mentions,Hashtags,Day,Month,Date,Time,Year,Pos,Neg,bin,Mentions_count,Hashtags_count,Mentions_score
731608,1236770340686069760,8676,1672,16,39,0,0,7,3,8,21,2020,3,-1,0,0,0,0
1355468,1251049144749154304,134387,147,18,168,0,0,5,4,17,7,2020,2,-1,0,0,0,0
547741,1226623357916995584,217727,230594,22,65,0,0,7,2,9,21,2020,1,-1,0,0,0,0
1707419,1248823243261136897,12024,175,17,53,IPRD_Bihar,BiharFightsCorona BiharCares,6,4,11,4,2020,2,-1,0,1,2,65
1063240,1239471429764231168,79264,76936,29,73,0,0,1,3,16,8,2020,1,-3,0,0,0,0


**Finding score of each row basing on hashtag popularity**

In [77]:
Hashtags_count = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Hashtags_count.csv')
Hashtags_count.set_index('Unnamed: 0', inplace=True)
Hashtags_count.head()

,0
Unnamed: 0,
COVID19,258958
coronavirus,205754
null;,99939
Coronavirus,52933
0,46187


In [78]:
def funct(string):
  if string == 0:
    return 0
  final = 0
  for i in string.split():
    final += Hashtags_count.loc[i].values[0]
  #print(string.split(), final)
  return final

In [79]:
frame['Hashtags_score'] = frame['Hashtags'].progress_apply(funct)

In [87]:
frame.head()

,Tweet Id,Followers,Friends,Retweets,Favorites,Mentions,Hashtags,Day,Month,Date,Time,Year,Pos,Neg,bin,Mentions_count,Hashtags_count,Mentions_score,Hashtags_score,Mentions_score_avg,Hashtags_score_avg
731608,1236770340686069760,8676,1672,16,39,0,0,7,3,8,21,2020,3,-1,0,0,0,0,0,0.0,0.0
1355468,1251049144749154304,134387,147,18,168,0,0,5,4,17,7,2020,2,-1,0,0,0,0,0,0.0,0.0
547741,1226623357916995584,217727,230594,22,65,0,0,7,2,9,21,2020,1,-1,0,0,0,0,0,0.0,0.0
1707419,1248823243261136897,12024,175,17,53,IPRD_Bihar,BiharFightsCorona BiharCares,6,4,11,4,2020,2,-1,0,1,2,65,167,65.0,83.5
1063240,1239471429764231168,79264,76936,29,73,0,0,1,3,16,8,2020,1,-3,0,0,0,0,0,0.0,0.0


In [84]:
frame['Mentions_score_avg'] = frame['Mentions_score']/frame['Mentions_count']
frame.Mentions_score_avg.replace(np.NaN,0,inplace=True)

In [86]:
frame['Hashtags_score_avg'] = frame['Hashtags_score']/frame['Hashtags_count']
frame.Hashtags_score_avg.replace(np.NaN,0,inplace=True)

In [88]:
frame.to_csv('Balanced.csv')
!cp Balanced.csv "drive/My Drive/Colab Notebooks/"